<a href="https://colab.research.google.com/github/StevenLomon/face-recognition-project/blob/CNN-model-development/face_recognition_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import imghdr
from imutils import paths
from matplotlib import pyplot as plt
#from tensorflow.keras.datasets import cifar10

# image preprocessing

In [ ]:
import shutil

data = pd.read_csv('fairface.csv')

main_directory = '/Users/yari/2023/Applicerad_AI/fairfacedata/fairface_images (kopia)'
structured_directory = '/Users/yari/2023/Applicerad_AI/fairfacedata/structured_images'

for index, row in data.iterrows():
    source = os.path.join(main_directory, row['file'])
    race_dir = os.path.join(structured_directory, row['race'])
    if not os.path.exists(race_dir):
        os.makedirs(race_dir)
    destination = os.path.join(race_dir, row['file'])
    if os.path.exists(source):
        shutil.move(source, destination)
    else:
        print(f'file could not be found and skipped: {source}')

In [2]:
data.head()

NameError: ignored

In [ ]:
images = tf.keras.utils.image_dataset_from_directory('/Users/yari/2023/Applicerad_AI/fairfacedata/structured_images')

Found 35424 files belonging to 7 classes.


In [ ]:
image_iterator = images.as_numpy_iterator()

In [ ]:

batch = image_iterator.next()

In [ ]:
#images represented as numpy array
batch[0].shape

(32, 256, 256, 3)

In [ ]:
batch[1] # this is the clauses i vår fall label for olika ras.

array([1, 6, 1, 5, 0, 2, 1, 0, 6, 6, 6, 1, 3, 0, 6, 1, 0, 5, 5, 4, 3, 3,
       3, 4, 3, 6, 6, 6, 1, 2, 5, 1], dtype=int32)

In [ ]:
# ska skala ner datan
scaled_images = images.map(lambda x, y: (x/255, y))

In [ ]:
scaled_image_iterator = scaled_images.as_numpy_iterator()

In [ ]:
batch = scaled_image_iterator.next()

In [ ]:
batch[0].min()

0.0

In [ ]:
fig, ax = plt.subplots(ncols=11, figsize = (20,20))
for idx, img in enumerate(batch[0][:11]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

# spliting data

In [ ]:
len(images)

1107

In [ ]:
train_size = int(len(images)*.7)
validate_size = int(len(images)*.2)
test_size = int(len(images)*.1)

In [ ]:
print(f'Train size: {train_size} \nTest size: {test_size} \nValidation size: {validate_size}')

Train size: 774 
Test size: 110 
Validation size: 221


In [ ]:
train = images.take(train_size)
test = images.skip(train_size).take(test_size)
validate = images.skip(train_size+test_size).take(validate_size)

# Modeling

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense , Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
# adding convolotional layer
model.add(Conv2D(16, (3,3),1, activation = 'relu', input_shape = (256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3),1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3),1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
import tensorflow as tf
model.compile('adam', loss= tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
log_directory= '/Users/yari/2023/Applicerad_AI/fairfacedata/logs'

In [ ]:
tensorbaord_callback = tf.keras.callbacks.TensorBoard(log_dir = log_directory)

In [ ]:
hist = model.fit(train, epochs=20, validation_data= validate, callbacks=[tensorbaord_callback])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color = 'teal', label = 'loss')
plt.plot(hist.history['val_accuracy'], color = 'orange', label = 'val_loss')
fig.suptitle('Loss', fontsize = 20)
plt.legend(loc = 'upper left')
plt.show()


In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color = 'teal', label = 'accuracy')
plt.plot(hist.history['val_accuracy'], color = 'orange', label = 'val_accuracy')
fig.suptitle('Accuracy', fontsize = 20)
plt.legend(loc = 'upper left')
plt.show()